In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import xarray 
import numpy as np
import logging
from netCDF4 import Dataset


hgrid_path="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/ocean_hgrid.nc"
topo_path="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/ocean_topog.nc"
ds_hgrid= xarray.open_dataset(hgrid_path)
ds_topo=xarray.open_dataset(topo_path)

print(ds_hgrid)  # Prints the variables, dimensions, and metadata
print(ds_topo)


<xarray.Dataset>
Dimensions:   (nyp: 1691, nxp: 1551, nx: 1550, ny: 1690)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 ...
    tile      |S255 ...
Attributes:
    history:  grid_hycom2mom6
<xarray.Dataset>
Dimensions:  (ntiles: 1, ny: 845, nx: 775)
Coordinates:
  * ntiles   (ntiles) int32 1
Dimensions without coordinates: ny, nx
Data variables:
    depth    (ny, nx) float32 ...
Attributes:
    grid_version:              0.2
    code_version:              ESMG/gridtools: ['heads/main-0-gea88375397', '']
    history:                   Thu Jan 13 17:05:42 2022: ncap2 -s defdim("nti...
    NCO:                       "4.5.4"
    nco_openmp_thread_number:  1


In [2]:


# # Downsampling factor
# factor = 5

# # Downsample `nyp` and `nxp` dimensions
# downsampled_nyp = slice(0, ds_hgrid.dims['nyp'], factor)
# downsampled_nxp = slice(0, ds_hgrid.dims['nxp'], factor)

# # Downsample `ny` and `nx` dimensions
# downsampled_ny = slice(0, ds_hgrid.dims['ny'], factor)
# downsampled_nx = slice(0, ds_hgrid.dims['nx'], factor)

# # Downsample the area variable by summing every 5x5 block
# coarsened_area = ds_hgrid['area'].coarsen(ny=factor, nx=factor, boundary="exact").sum()

# # Downsample the dataset for other variables
# downsampled_ds_hgrid = ds_hgrid.isel(
#     nyp=downsampled_nyp,
#     nxp=downsampled_nxp,
#     ny=downsampled_ny,
#     nx=downsampled_nx
# )

# # Replace the area variable with the adjusted coarsened area
# downsampled_ds_hgrid['area'] = coarsened_area

# # Save the downsampled dataset to a new NetCDF file
# output_file = "/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_hgrid_gene.nc"  # Replace with the actual output path
# downsampled_ds_hgrid.to_netcdf(output_file)

# # Verify the total area consistency
# original_area_sum = ds_hgrid['area'].sum().item()
# adjusted_area_sum = coarsened_area.sum().item()

# print(f"Original total area: {original_area_sum}")
# print(f"Adjusted total area: {adjusted_area_sum}")


# # Trim dimensions to ensure they are divisible by the coarsening factor
# factor = 5

# # Apply coarsening along ny and nx with averaging
# coarsened_ds_topo = ds_topo.coarsen(ny=factor, nx=factor, boundary="exact").mean()


# # Reorder dimensions explicitly
# coarsened_ds_topo = coarsened_ds_topo.transpose("ntiles", "ny", "nx")
# # Save the downsampled dataset to a new NetCDF file
# output_file = "/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_topog_gene.nc"  # Replace with the desired output file path
# coarsened_ds_topo.to_netcdf(output_file)

# print(downsampled_ds_hgrid)
# print(coarsened_ds_topo)


In [125]:
ds_topo

<xarray.Dataset>
Dimensions:  (ntiles: 1, ny: 845, nx: 775)
Coordinates:
  * ntiles   (ntiles) int32 1
Dimensions without coordinates: ny, nx
Data variables:
    depth    (ny, nx) float32 ...
Attributes:
    grid_version:              0.2
    code_version:              ESMG/gridtools: ['heads/main-0-gea88375397', '']
    history:                   Thu Jan 13 17:05:42 2022: ncap2 -s defdim("nti...
    NCO:                       "4.5.4"
    nco_openmp_thread_number:  1

In [4]:
original_area = ds_hgrid['area']

# Truncate dimensions to ensure even values before downsampling
# Truncate dimensions to ensure even values before downsampling
nyp_truncated = 845 * 2
nxp_truncated = 775 * 2
ny_truncated = 844 * 2
nx_truncated = 774 * 2


truncated_area = original_area.isel(
    ny=slice(0, ny_truncated),
    nx=slice(0, nx_truncated)
)

# Identify the trimmed rows/columns and save their contributions
trimmed_rows_area = original_area.isel(ny=slice(ny_truncated, None), nx=slice(0, nx_truncated)).sum(dim='ny')
trimmed_cols_area = original_area.isel(ny=slice(0, ny_truncated), nx=slice(nx_truncated, None)).sum(dim='nx')
trimmed_corner_area = original_area.isel(ny=slice(ny_truncated, None), nx=slice(nx_truncated, None)).sum()

# Downsample the truncated area by taking every other point
coarsened_area = truncated_area.coarsen(ny=2, nx=2, boundary='trim').sum()

# Add the trimmed rows' areas to the bottom row of the coarsened grid
if trimmed_rows_area.size > 0:
    coarsened_area[-1, :] += trimmed_rows_area.coarsen(nx=2, boundary='trim').sum()

# Add the trimmed columns' areas to the rightmost column of the coarsened grid
if trimmed_cols_area.size > 0:
    coarsened_area[:, -1] += trimmed_cols_area.coarsen(ny=2, boundary='trim').sum()

# Add the trimmed corner area to the bottom-right cell
if trimmed_corner_area.size > 0:
    coarsened_area[-1, -1] += trimmed_corner_area

# Replace the downsampled area in the new dataset

downsampled_ds_hgrid = ds_hgrid.isel(
    nyp=slice(0, nyp_truncated, 2),
    nxp=slice(0, nxp_truncated, 2),
    ny=slice(0, ny_truncated, 2),
    nx=slice(0, nx_truncated, 2)
)
downsampled_ds_hgrid['area'] = coarsened_area

# Save the adjusted downsampled dataset
output_file = '/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_hgrid.nc'
downsampled_ds_hgrid.to_netcdf(output_file)

# Verify the total area consistency
original_area_sum = original_area.sum().item()
adjusted_downsampled_area_sum = coarsened_area.sum().item()
print(f"Original total area: {original_area_sum}")
print(f"Adjusted downsampled total area: {adjusted_downsampled_area_sum}")


ds_trimmed_topo = ds_topo.isel(
    ny=slice(0, ds_topo.dims['ny'] - (ds_topo.dims['ny'] % 2)),  # Ensure 'ny' is even
    nx=slice(0, ds_topo.dims['nx'] - (ds_topo.dims['nx'] % 2))   # Ensure 'nx' is even
)

# Downsample by selecting every other point
# downsampled_ds_topo = ds_trimmed_topo.isel(
#     ny=slice(0, None, 2),  # Take every second point along 'ny'
#     nx=slice(0, None, 2)   # Take every second point along 'nx'
# )

downsampled_ds_topo = ds_trimmed_topo.coarsen(ny=2, nx=2, boundary='trim').mean()
output_file="/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_topog.nc"
# Save the downsampled dataset to a new NetCDF file
downsampled_ds_topo.to_netcdf(output_file)
downsampled_ds_topo
print(f"Downsampled dataset saved to {output_file}")


Original total area: 31735230937005.496
Adjusted downsampled total area: 31735230937005.504
Downsampled dataset saved to /gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_topog.nc


In [121]:
truncated_area

<xarray.DataArray 'area' (ny: 1688, nx: 1548)>
[2613024 values with dtype=float64]
Dimensions without coordinates: ny, nx
Attributes:
    units:    m2

In [122]:
# import numpy as np
# import xarray as xr

# # Access the original area
# original_area = ds_hgrid['area']

# # Original dimensions
# original_nyp, original_nxp = ds_hgrid.dims['nyp'], ds_hgrid.dims['nxp']
# original_ny, original_nx = ds_hgrid.dims['ny'], ds_hgrid.dims['nx']

# # Calculate the middle indices for truncation
# middle_nyp = original_nyp // 2
# middle_nxp = original_nxp // 2
# middle_ny = original_ny // 2
# middle_nx = original_nx // 2

# # Truncate the dimensions to skip middle rows/columns
# truncated_area = original_area.isel(
#     ny=np.r_[:middle_ny, middle_ny + 2:original_ny],  # Skip the middle row
#     nx=np.r_[:middle_nx, middle_nx + 2:original_nx]   # Skip the middle column
# )

# # Identify the trimmed areas for the removed row and column
# trimmed_row_area = original_area.isel(ny=[middle_ny,middle_ny+1], nx=slice(0, original_nx)).sum(dim='ny')
# trimmed_col_area = original_area.isel(ny=slice(0, original_ny), nx=[middle_nx,middle_nx+1]).sum(dim='nx')
# trimmed_corner_area = original_area.isel(ny=[middle_ny,middle_ny+1], nx=[middle_nx,middle_nx+1]).sum()

# # Downsample the truncated area, keeping the dimensions intact
# coarsened_area = truncated_area.coarsen(ny=2, nx=2, boundary='trim').sum()

# # Compute integer indices for placing the trimmed areas back
# coarsened_middle_ny = middle_ny // 2
# coarsened_middle_nx = middle_nx // 2

# if trimmed_row_area.size > 0:
#     # Directly insert the trimmed row area back into the coarsened grid
#     coarsened_area.loc[{'ny': coarsened_middle_ny, 'nx': slice(None)}] += trimmed_row_area.coarsen(nx=2, boundary='trim').sum()[:-1]

# # Add the trimmed column area back to the coarsened grid at the corresponding column index
# if trimmed_col_area.size > 0:
#     # Directly insert the trimmed column area back into the coarsened grid
#     coarsened_area.loc[{'ny': slice(None), 'nx': coarsened_middle_nx}] += trimmed_col_area.coarsen(ny=2, boundary='trim').sum()[:-1]

# # Add the trimmed corner area back to the coarsened grid at the corresponding corner
# if trimmed_corner_area.size > 0:
#     # Directly insert the trimmed corner area back into the coarsened grid
#     coarsened_area.loc[{'ny': coarsened_middle_ny, 'nx': coarsened_middle_nx}] -= trimmed_corner_area

# truncated_ds_hgrid = ds_hgrid.isel(
#     nyp=np.r_[:middle_nyp, middle_nyp + 1:original_nyp],  # Removing middle rows
#     nxp=np.r_[:middle_nxp, middle_nxp + 1:original_nxp],  # Removing middle columns
#     ny=np.r_[:middle_ny, middle_ny + 2:original_ny],     # Removing middle rows
#     nx=np.r_[:middle_nx, middle_nx + 2:original_nx]      # Removing middle columns

# )

# # Step 2: Downsample by selecting every second point along each dimension
# downsampled_ds_hgrid = truncated_ds_hgrid.isel(
#     nyp=slice(None, None, 2),  # Select every second row
#     nxp=slice(None, None, 2),  # Select every second column
#     ny=slice(None, None, 2),   # Select every second row
#     nx=slice(None, None, 2)    # Select every second column
# )
# # Replace the area variable with the adjusted coarsened area
# downsampled_ds_hgrid['area'] = coarsened_area

# # Save the adjusted downsampled dataset
# output_file = '/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling_ZC/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_hgrid_gene.nc'
# downsampled_ds_hgrid.to_netcdf(output_file)

# # Verify the total area consistency
# original_area_sum = original_area.sum().item()
# adjusted_downsampled_area_sum = coarsened_area.sum().item()

# print(f"Original total area: {original_area_sum}")
# print(f"Adjusted downsampled total area: {adjusted_downsampled_area_sum}")

# # Process the topography data similarly
# original_topo = ds_topo['depth']

# # Original dimensions
# ny, nx = ds_topo.dims['ny'], ds_topo.dims['nx']

# # Calculate the middle indices for truncation
# middle_ny = ny // 2
# middle_nx = nx // 2

# # Truncate the dimensions to skip middle rows/columns
# truncated_topo = original_topo.isel(
#     ny=np.r_[:middle_ny, middle_ny + 1:ny],  # Skip the middle row
#     nx=np.r_[:middle_nx, middle_nx + 1:nx]   # Skip the middle column
# )

# # Downsample the truncated data (every second point for ny and nx)
# downsampled_topo = truncated_topo.coarsen(ny=2, nx=2, boundary='trim').mean()

# # Verify the dimensions to ensure that ntiles is preserved
# print(f"Original shape: {original_topo.shape}")
# print(f"Downsampled shape: {downsampled_topo.shape}")

# # Save the downsampled topography dataset
# output_file = '/gpfs/f5/gfdl_o/scratch/Qian.Xiao/FMS_Wave_Coupling_ZC/examples/NWA12.COBALT_all_wave1_6/INPUT/EXCHANGE_GRIDS/wave_topog_gene.nc'
# downsampled_topo.to_netcdf(output_file)






Original total area: 31735230937005.496
Adjusted downsampled total area: 31735162093303.473
Original shape: (845, 775)
Downsampled shape: (422, 387)


In [126]:
downsampled_ds_hgrid

<xarray.Dataset>
Dimensions:   (nyp: 845, nxp: 775, nx: 774, ny: 844)
Dimensions without coordinates: nyp, nxp, nx, ny
Data variables:
    x         (nyp, nxp) float64 ...
    y         (nyp, nxp) float64 ...
    angle_dx  (nyp, nxp) float64 ...
    dx        (nyp, nx) float64 ...
    dy        (ny, nxp) float64 ...
    area      (ny, nx) float64 7.865e+07 7.862e+07 ... 2.293e+07 2.294e+07
    tile      |S255 ...
Attributes:
    history:  grid_hycom2mom6

In [37]:
area_6=downsampled_ds_hgrid.variables['area']
x_6=downsampled_ds_hgrid.variables['x']
y_6=downsampled_ds_hgrid.variables['y']
depth_6=downsampled_ds_topo['depth'].values
print(area_6[:10,:10].values)
print("above area")
print(x_6[:10,:10].values)
print("above x value")
print(y_6[:10,:10].values)
print("above y value")
print(area_6.sum().item())
print(depth_6[10:,10:])
print("above depth")

[[78652549.56161022 78622552.40074682 78652549.56161022 78622552.40074682
  78622552.40074682 78652549.56161022 78622552.40074682 78652549.56161022
  78622552.40074682 78652549.56161022]
 [78632497.01365471 78602507.50060034 78632497.01365471 78602507.50060034
  78602507.50060034 78632497.01365471 78602507.50060034 78632497.01365471
  78602507.50060034 78632497.01365471]
 [78611355.07377625 78581373.62401056 78611355.07377625 78581373.62401056
  78581373.62401056 78611355.07377625 78581373.62401056 78611355.07377625
  78581373.62401056 78611355.07377625]
 [78591006.70753098 78561033.01839685 78591006.70753098 78561033.01839685
  78561033.01839685 78591006.70753098 78561033.01839685 78591006.70753098
  78561033.01839685 78591006.70753098]
 [78569569.47734451 78539603.96411991 78569569.47734451 78539603.96411991
  78539603.96411991 78569569.47734451 78539603.96411991 78569569.47734451
  78539603.96411991 78569569.47734451]
 [78547984.9424839  78518027.66135025 78547984.9424839  78518027.

In [38]:

area_12=ds_hgrid.variables['area']
depth_12=ds_topo['depth'].values
x_12=ds_hgrid.variables['x']
y_12=ds_hgrid.variables['y']
print(area_12[:10,:10].values)
print("above area")
print(x_12[:10,:10].values)
print("above x")
print(y_12[:10,:10].values)
print("above y")
print(area_12.sum().item())
print(depth_12[10:,10:])
print("above depth")

[[19664945.11717176 19664945.11717176 19649945.15784836 19664945.11717176
  19664945.11717176 19664945.11717176 19649945.15784836 19664945.11717176
  19664945.11717176 19649945.15784836]
 [19661329.66363335 19661329.66363335 19646332.46209335 19661329.66363335
  19661329.66363335 19661329.66363335 19646332.46209335 19661329.66363335
  19661329.66363335 19646332.46209335]
 [19660058.85293341 19660058.85293341 19645062.62073803 19660058.85293341
  19660058.85293341 19660058.85293341 19645062.62073803 19660058.85293341
  19660058.85293341 19645062.62073803]
 [19656189.65389395 19656189.65389395 19641196.37303495 19656189.65389395
  19656189.65389395 19656189.65389395 19641196.37303495 19656189.65389395
  19656189.65389395 19641196.37303495]
 [19654900.18478441 19654900.18478441 19639907.88750076 19654900.18478441
  19654900.18478441 19654900.18478441 19639907.88750076 19654900.18478441
  19654900.18478441 19639907.88750076]
 [19650777.35210371 19650777.35210371 19635788.19962168 19650777.